In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    '_ga': 'GA1.2.2139615999.1684772915',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.2.2139615999.1684772915',
    'Pragma': 'no-cache',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

In [4]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies,
    headers=headers,)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [5]:
files = glob('pages/*.json')
len(files)

18787

In [6]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [7]:
filtered_pdfs = []
for p in tqdm(pdfs):
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
    if 'ade.sagepub.com' in p:
        continue
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|███████████████████████████████| 235433/235433 [00:00<00:00, 639115.93it/s]


235130

In [8]:
usm = [f for f in filtered_pdfs if 'usm.my' in f]
len(usm)

35731

In [9]:
# !rm -rf usm
!mkdir usm

mkdir: cannot create directory ‘usm’: File exists


In [10]:
max_worker = 1

usm = sorted(list(usm))
for i in tqdm(range(0, len(usm), max_worker)):
    urls = [(url, os.path.join('usm', f'{i}-{no}.pdf')) for no, url in enumerate(usm[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  2%|▊                                      | 763/35731 [00:28<19:33, 29.81it/s]

0 HTTPConnectionPool(host='eprints.usm.my', port=80): Max retries exceeded with url: /1285/1/YSF_512_-_SEJARAH_FILEM_SEPT.02.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f57687e9820>: Failed to establish a new connection: [Errno 110] Connection timed out'))


  3%|█                                   | 1100/35731 [03:41<2:55:05,  3.30it/s]

0 HTTPConnectionPool(host='eprints.usm.my', port=80): Max retries exceeded with url: /13757/1/EBS_215-3_-_COMMINUTION_AND_SIZING_APR-MEI_09.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5768785a00>: Failed to establish a new connection: [Errno 110] Connection timed out'))


  3%|█                                 | 1103/35731 [05:54<188:50:55, 19.63s/it]

...

100%|█████████████████████████████████▉| 35634/35731 [30:05:17<00:43,  2.23it/s]

0 HTTPConnectionPool(host='web.usm.my', port=80): Max retries exceeded with url: /jcdc/vol19_1_2014/JCDC%2019(1)%202014-Art.%203%20(35-52).pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5768799f10>: Failed to establish a new connection: [Errno 110] Connection timed out'))


100%|█████████████████████████████████▉| 35710/35731 [30:07:55<00:08,  2.35it/s]

0 HTTPConnectionPool(host='web.usm.my', port=80): Max retries exceeded with url: /journal/aamjaf/aamjaf13012017/aamjaf13012017_6.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f576873bca0>: Failed to establish a new connection: [Errno 110] Connection timed out'))


100%|██████████████████████████████████| 35731/35731 [30:10:17<00:00,  3.04s/it]
